In [1]:
import os

# 获取当前工作路径
current_dir = os.getcwd()
print("当前工作路径：", current_dir)
# 指定新的工作目录路径
# new_working_directory = r"C:\Users\REBECCA329\Desktop\BIO\file\Data\Data\1.DLPFC\151672\pythonProject2\GraphST-main"
# 
# # 更改工作目录
# os.chdir(new_working_directory)
# 
# # 确认工作目录已更改
# print("当前工作目录:", os.getcwd())


当前工作路径： E:\JSU\BIO\file\GraphST-main\GraphST-main\GraphST


In [2]:
import os
import torch
import pandas as pd
import scanpy as sc
from sklearn import metrics
import multiprocessing as mp
from GraphST import GraphST  #从模块导入类



ModuleNotFoundError: No module named 'ot'

In [3]:
# Run device, by default, the package is implemented on 'cpu'. We recommend using GPU.
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
# the location of R, which is necessary for mclust algorithm. Please replace the path below with local R installation path
os.environ['R_HOME'] = 'F:/Program Files/R/R-4.3.2'

# the number of clusters
n_clusters = 5
dataset = '151672'
#读取ST数据
# read data
file_fold = r'C:\Users\REBECCA329\Desktop\BIO\file\Data\Data\1.DLPFC/' + str(
    dataset)  #please replace 'file_fold' with the download path
print(file_fold)
adata = sc.read_visium(file_fold, count_file='filtered_feature_bc_matrix.h5', load_images=True)

adata.var_names_make_unique()
adata


cuda:0
C:\Users\REBECCA329\Desktop\BIO\file\Data\Data\1.DLPFC/151672


FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = 'C:\Users\REBECCA329\Desktop\BIO\file\Data\Data\1.DLPFC\151672\filtered_feature_bc_matrix.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
#训练模型、
# define model
model = GraphST.GraphST(adata, device=device)  #似乎应该在后面一个步骤中去实现

# train model
adata = model.train()
adata
#空间聚类和细化
# set radius to specify the number of neighbors considered during refinement
radius = 50

tool = 'mclust'  # mclust, leiden, and louvain

# clustering
from GraphST.utils import clustering

if tool == 'mclust':
    clustering(adata, n_clusters, radius=radius, method=tool,
               refinement=True)  # For DLPFC dataset, we use optional refinement step.
elif tool in ['leiden', 'louvain']:
    clustering(adata, n_clusters, radius=radius, method=tool, start=0.1, end=2.0, increment=0.01, refinement=False)

#可视化
# add ground_truth
df_meta = pd.read_csv(file_fold + '/metadata.tsv', sep='\t')
df_meta_layer = df_meta['layer_guess_reordered']
adata.obs['ground_truth'] = df_meta_layer.values  #转化成numpy数组

# filter out NA nodes
adata = adata[~pd.isnull(adata.obs['ground_truth'])]

# calculate metric ARI
ARI = metrics.adjusted_rand_score(adata.obs['domain'], adata.obs['ground_truth'])
adata.uns['ARI'] = ARI

print('Dataset:', dataset)
print('ARI:', ARI)

# plotting spatial clustering result
sc.pl.spatial(adata,
              img_key="hires",
              color=["ground_truth", "domain"],
              title=["Ground truth", "ARI=%.4f" % ARI],
              show=True)  #color表示用来着色的变量

# plotting predicted labels by UMAP
sc.pp.neighbors(adata, use_rep='emb_pca', n_neighbors=10)
sc.tl.umap(adata)
sc.pl.umap(adata, color='domain', title=['Predicted labels'], show=False)
# plotting predicted labels by UMAP
sc.pp.neighbors(adata, use_rep='emb_pca', n_neighbors=10)
sc.tl.umap(adata)
sc.pl.umap(adata, color='domain', title=['Predicted labels'], show=False)

